<a href="https://colab.research.google.com/github/ModernOctave/Thermal-Image-Human-Detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependancies


In [ ]:
# Import Tensorflow 2.0
import tensorflow as tf 
import numpy as np
import cv2 as cv
import glob
import random

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
# assert len(tf.config.list_physical_devices('GPU')) > 0

# Create Dataset

In [11]:
def createDataset():
    human_paths = glob.glob('./dataset/human/*.jpg')
    no_human_paths = glob.glob('./dataset/no_human/*.jpg')
    if len(human_paths) == 0 or len(no_human_paths) == 0:
        raise Exception('Dataset could not be found!')
    human_entries = list(zip([cv.imread(path) for path in human_paths], [1 for _ in human_paths]))
    no_human_entries = list(zip([cv.imread(path) for path in no_human_paths], [0 for _ in no_human_paths]))
    random.shuffle(human_entries)
    random.shuffle(no_human_entries)
    half_size = min(len(human_entries), len(no_human_entries))
    dataset = human_entries[:half_size] + no_human_entries[:half_size]
    random.shuffle(dataset)
    split_size = int(len(dataset) * 0.8)
    train_dataset = dataset[:split_size]
    test_dataset = dataset[split_size:]
    return train_dataset, test_dataset

train_dataset, test_dataset = createDataset()

# Model


## Create the model

In [ ]:
def build_cnn_model():
    cnn_model = tf.keras.Sequential([
                                     
        tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation=tf.nn.relu, input_shape=(24, 32, 1)),

        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.nn.relu),

        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation=tf.nn.relu),

        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(units=64, activation=tf.nn.relu),

        tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid),
    ])
    
    return cnn_model
  
cnn_model = build_cnn_model()

# Initialize the model by passing some data through
# cnn_model.predict(train_images[[0]])

# Compile the model
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the summary of the layers in the model.
print(cnn_model.summary())

## Train the model

In [ ]:
cnn_model = tf.keras.models.load_model("yolo_lite.h5")

# Print the summary of the layers in the model.
print(cnn_model.summary())

In [ ]:
cnn_model.fit(train_images, train_labels, epochs=5)

In [ ]:
accuracy = 0.01
last_accuracy = 0
while accuracy > last_accuracy:
  cnn_model.fit(train_images, train_labels, epochs=1)
  loss, accuracy = cnn_model.evaluate(test_images, test_labels, batch_size=1)
print(round(accuracy,2))

In [ ]:
cnn_model.save('yolov3-tiny.h5')

In [ ]:
tf.saved_model.save(cnn_model, "yolo_lite")

## Test the model


In [ ]:
loss, accuracy = cnn_model.evaluate(test_images, test_labels, batch_size=1)